# Rollup of Total Grant Value for all US Counties

Grant information by state was compiled by [Investigate West](http://www.invw.org/2012/06/11/lwcf-grants-database-1283/) for the period 1965-2011.

In [1]:
import glob
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Collect all file names and process into DataFrames
state_files = glob.glob('./Data/StateGrantData/*.xlsx')
dfs = []

for file in state_files:
    dfs.append(pd.read_excel(file))

all_states = pd.concat(dfs, ignore_index=True)
all_states.head()

,Grant ID & Element,Grant Name,Sponsor,County,State,Grant Amount,Year Approved,Year Completed,Type
0,3 - XXX,INDIAN PEAKS - BAKER CANYON,STATE OF UTAH,BEAVER,UT,5985.92,1966,1966,Development
1,4 - XXX,WEST BEAR RIVER BAY,STATE OF UTAH,BOX ELDER,UT,18500.00,1966,1967,Development
2,6 - XXX,WELLSVILLE PARK,CITY OF WELLSVILLE,CACHE,UT,934.25,1966,1967,Acquisition
3,2 - XXX,FARMINGTON BAY WATERFOWL AREA,STATE OF UTAH,DAVIS,UT,15150.00,1966,1967,Acquisition
4,11 - XXX,4 CITY PARKS,CITY OF BOUNTIFUL,DAVIS,UT,16225.15,1966,1966,Development


In [3]:
all_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41145 entries, 0 to 41144
Data columns (total 9 columns):
Grant ID & Element    41145 non-null object
Grant Name            41145 non-null object
Sponsor               41145 non-null object
County                41145 non-null object
State                 41145 non-null object
Grant Amount          41145 non-null float64
Year Approved         41145 non-null int64
Year Completed        41145 non-null int64
Type                  41145 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 2.8+ MB


In [25]:
print('Total grant value: ${0:,.0f}'.format(all_states['Grant Amount'].sum()))
print('Total grants: {0:,.0f}'.format(all_states['Grant Amount'].count()))

Total grant value: $3,821,751,060
Total grants: 41,145


In [5]:
all_states['State'].sort_values().unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [6]:
# Count states - should be 51 (includes DC)
all_states['State'].nunique()

51

In [7]:
# Fix structural issues in county names to tie to county id data

# Make all counties lowercase
all_states['County'] = all_states['County'].map(lambda s: s.lower())

# Change all occurences of "saint" to "st."
all_states['County'] = all_states['County'].map(lambda s: s.replace('saint', 'st.'))

## Group County Data

In [8]:
# Create grouped grant data
all_counties = all_states.groupby(['State', 'County'])['Grant Amount'].agg(['count', 'sum'])

# Reset multi-index to flatten columns
all_counties = all_counties.reset_index()

# Re-name columns
all_counties.columns = ['state', 'county_orig', 'num_grants', 'grant_value']

# Round grant_value column to whole dollars, convert type to int
all_counties['grant_value'] = all_counties['grant_value'].map(lambda n: int(round(n)))

all_counties.head()

,state,county_orig,num_grants,grant_value
0,AK,aleutians east,5,123553
1,AK,aleutians west,1,52405
2,AK,anchorage,67,9475507
3,AK,bethel,3,366133
4,AK,dillingham,2,96120


In [9]:
# Drop multi-county, planning, and no county rows
drop_rows = ['multi-county', 'planning', 'no county', 'county name missing']
all_counties = all_counties[~all_counties['county_orig'].isin(drop_rows)]

In [10]:
all_counties[all_counties['state'] == 'RI']

,state,county_orig,num_grants,grant_value
2280,RI,bristol,17,2504765
2281,RI,kent,52,4462590
2283,RI,newport,35,4809098
2285,RI,providence,139,13877173
2286,RI,washington,75,12637118


In [11]:
all_counties['county_orig'].count()

2946

In [12]:
# Create key column to merge on
all_counties['county_key'] = all_counties['county_orig'] + ' ' + all_counties['state']
all_counties.head()

,state,county_orig,num_grants,grant_value,county_key
0,AK,aleutians east,5,123553,aleutians east AK
1,AK,aleutians west,1,52405,aleutians west AK
2,AK,anchorage,67,9475507,anchorage AK
3,AK,bethel,3,366133,bethel AK
4,AK,dillingham,2,96120,dillingham AK


## Process County ID Data

In [13]:
# Import county id data
county_ids = pd.read_csv('./Data/county_names.csv', sep=',', dtype=str)
county_ids.head()

,id,state,county
0,01001,Alabama,Autauga County
1,01003,Alabama,Baldwin County
2,01005,Alabama,Barbour County
3,01007,Alabama,Bibb County
4,01009,Alabama,Blount County


In [14]:
county_ids['county'].count()

3219

In [15]:
# Create state name and abbrev dictionary
st_name = county_ids['state'].unique()
st_abbrev = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
             'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
             'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
             'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
             'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR']


state_dict = dict(zip(st_name, st_abbrev))

In [16]:
# Add state abbreviation column
county_ids['state_abbv'] = county_ids['state'].map(lambda s: state_dict[s])
county_ids.head()

,id,state,county,state_abbv
0,01001,Alabama,Autauga County,AL
1,01003,Alabama,Baldwin County,AL
2,01005,Alabama,Barbour County,AL
3,01007,Alabama,Bibb County,AL
4,01009,Alabama,Blount County,AL


In [17]:
# Edit county names to tie to grant data county names, create key column to merge on
county_ids['county_key'] = county_ids['county'].map(lambda s: s.lower())

pattern = re.compile(r' county| census area| municipality| borough| city and borough| parish')
county_ids['county_key'] = county_ids['county_key'].str.replace(pattern, '')

county_ids['county_key'] = county_ids['county_key'] + ' ' + county_ids['state_abbv']

county_ids[county_ids['state_abbv'] == 'RI']

,id,state,county,state_abbv,county_key
2310,44001,Rhode Island,Bristol County,RI,bristol RI
2311,44003,Rhode Island,Kent County,RI,kent RI
2312,44005,Rhode Island,Newport County,RI,newport RI
2313,44007,Rhode Island,Providence County,RI,providence RI
2314,44009,Rhode Island,Washington County,RI,washington RI


In [18]:
# Drop unnecessary columns for merge
county_ids.drop(['state', 'county', 'state_abbv'], axis=1, inplace=True)
county_ids.head()

,id,county_key
0,01001,autauga AL
1,01003,baldwin AL
2,01005,barbour AL
3,01007,bibb AL
4,01009,blount AL


## Merge Grant and County ID DataFrames

In [19]:
all_data = pd.merge(all_counties, county_ids, how='left', on='county_key')

all_data[all_data['state'] == 'AK']

,state,county_orig,num_grants,grant_value,county_key,id
0,AK,aleutians east,5,123553,aleutians east AK,02013
1,AK,aleutians west,1,52405,aleutians west AK,02016
2,AK,anchorage,67,9475507,anchorage AK,02020
3,AK,bethel,3,366133,bethel AK,02050
4,AK,dillingham,2,96120,dillingham AK,02070
5,AK,fairbanks north star,23,2765459,fairbanks north star AK,02090
6,AK,haines,10,2978362,haines AK,02100
7,AK,juneau,18,1945648,juneau AK,02110
8,AK,kenai peninsula,35,3052184,kenai peninsula AK,02122
9,AK,ketchikan gateway,9,809122,ketchikan gateway AK,02130


In [20]:
# Check for null values in id column
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2946 entries, 0 to 2945
Data columns (total 6 columns):
state          2946 non-null object
county_orig    2946 non-null object
num_grants     2946 non-null int64
grant_value    2946 non-null int64
county_key     2946 non-null object
id             2946 non-null object
dtypes: int64(2), object(4)
memory usage: 161.1+ KB


In [21]:
# Create simple DataFrame to export to json
grants_by_county = all_data.drop(['county_key'], axis=1)
grants_by_county.columns = ['state', 'county', 'num_grants', 'grant_value', 'id']
grants_by_county.set_index('id', inplace=True)

grants_by_county.head()

,state,county,num_grants,grant_value
id,,,,
02013,AK,aleutians east,5,123553
02016,AK,aleutians west,1,52405
02020,AK,anchorage,67,9475507
02050,AK,bethel,3,366133
02070,AK,dillingham,2,96120


In [22]:
print('Min county grant value: ${0:,.0f}'.format(grants_by_county['grant_value'].min()))
print('Max county grant value: ${0:,.0f}'.format(grants_by_county['grant_value'].max()))
print('Total LWCF by county: ${0:,.0f}'.format(grants_by_county['grant_value'].sum()))

Min county grant value: $443
Max county grant value: $58,641,286
Total LWCF by county: $3,512,205,989


In [23]:
grants_by_county['grant_value'].describe()

count    2.946000e+03
mean     1.192195e+06
std      2.654758e+06
min      4.430000e+02
25%      1.580910e+05
50%      4.056090e+05
75%      1.067659e+06
max      5.864129e+07
Name: grant_value, dtype: float64

In [24]:
# Export data to JSON to use to create interactive map
# grants_by_county.to_json('./GrantMap/us_grants_by_county.json', orient='index')